In [1]:
import os
import data_loader as dl
import correlation_plot as cp
import profile_plot as dp
import express_statistic_plot as es
import trf_type_barplot as tt
import read_statistic as rs
import pathlib
import pylab


%matplotlib inline

'''
# Visualization module of tRNAExploer (v1.0) 
# The visual configure file
'''
#visual_config = "/Users/hqyone/PycharmProjects/data_output/server2/visual_config.tsv"
visual_config = "/Users/hqyone/PycharmProjects/data_output/tRNA_seq/output/visual_config.tsv"
#visual_config = "/Users/hqyone/PycharmProjects/data_output/cancer/output/visual_config.tsv"
wdir = '/media/hqyone/3T/trna/data/trna/trna_test_out'
visual_config=wdir+'/visual_config.tsv'

if not os.path.isfile(visual_config):
    print("Can't find config file. Abort !")
    exit(-1)

# If you don't set report_dir, the default path will be <output_dir>+"/reports"
report_dir = ""

# Loading data
d =  dl.LoadConfig(visual_config, report_dir)

def csv_download_link(df, csv_file_name, delete_prompt=True):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    df.to_csv(csv_file_name, index=True, sep='\t')
    from IPython.display import FileLink
    display(FileLink(csv_file_name))
    if delete_prompt:
        a = input('Press enter to delete the file after you have downloaded it.')
        import os
        os.remove(csv_file_name)

/Users/hqyone/PycharmProjects/data_output/tRNA_seq/output/static.log


In [ ]:
import numpy as np
import pandas as pd
import re

# The function is used to selected o

In [2]:
import numpy as np
import pandas as pd
import re

def CombineProfilesWithIntron(sample , tRNA1_ID, tRNA2_ID, d, profile_type):
    p_df = pd.read_csv(d["profiles"], sep="\t")
    st_df = d["st_df"]
    print(st_df.columns)
    t1_df = p_df[(p_df['tRNA_ID'] == tRNA1_ID) & (p_df['#SampleID'] == sample) & (p_df['type'] == profile_type)]
    t2_df = p_df[(p_df['tRNA_ID'] == tRNA2_ID) & (p_df['#SampleID'] == sample) & (p_df['type'] == profile_type)]
    profile_1 = np.array(t1_df['profile'].values[0].split(",")).astype(np.float)
    profile_2 = np.array(t2_df['profile'].values[0].split(",")).astype(np.float)
    
    intron_1_str = st_df[st_df['name']==tRNA1_ID]['intron_infor'].values[0]
    print(intron_1_str)
    intron_1_obj = {}
    m = re.search(r'\((\d+)-(\d+)\)', intron_1_str)
    if m:
        intron_1_obj['start'] = int(m.group(1))
        intron_1_obj['end'] = int(m.group(2))
    print(intron_1_str)
    intron_2_str = st_df[st_df['name']==tRNA2_ID]['intron_infor'].values[0]
    intron_2_obj = {}
    m = re.search(r'\((\d+)-(\d+)\)', intron_2_str)
    if m:
        intron_2_obj['start'] = int(m.group(1))
        intron_2_obj['end'] = int(m.group(2))
    
    index1=0
    index2=0
    for i in range(0,len(profile_1)):
        if index1==intron_1_obj['start']-1:
            index1=intron_1_obj['end']
        if index2==intron_2_obj['start']-1:
            index2=intron_2_obj['end']
        if index1<len(profile_1) and index2<len(profile_2):
            profile_1[index1]+= profile_2[index2]
        index1+=1
        index2+=1
    return profile_1

p_df = pd.read_csv(d["profiles"], sep="\t")
sample_ls = p_df["#SampleID"]
print(sample_ls)
print(CombineProfilesWithIntron('SRR1836129' , 'tRNA-Leu-CAA-1-1', 'tRNA-Leu-CAA-1-2', d, 'total'))

0        SRR1836129
1        SRR1836129
2        SRR1836129
3        SRR1836129
4        SRR1836129
            ...    
16560    SRR1836127
16561    SRR1836127
16562    SRR1836127
16563    SRR1836127
16564    SRR1836127
Name: #SampleID, Length: 16565, dtype: object
Index(['#chrom', 'start', 'end', 'name', 'score', 'strand', 'full_seq', 'seq',
       'intron_infor', 'anticodon', 'anticodon_start', 'anticodon_end',
       'acceptor', 'family', 'seq_utr', 'utr_len', 'map_start', 'map_end',
       'map_len', 'map_structure_str', 'map_seq', 'anticodon_start_in_map',
       'anticodon_end_in_map', 'map_scan_score', 'possible_type',
       'd_loop_start', 'd_loop_lstart', 'd_loop_lend', 'd_loop_end',
       'a_loop_start', 'a_loop_lstart', 'a_loop_lend', 'a_loop_end',
       'v_loop_start', 'v_loop_lstart', 'v_loop_lend', 'v_loop_end',
       't_loop_start', 't_loop_lstart', 't_loop_lend', 't_loop_end',
       'stem_for_start', 'stem_for_end', 'stem_rev_start', 'stem_rev_end'],
      dtype='o

In [ ]:
import seaborn as sns
import clusterMatrix as cm

sns.set()

var = d['variants']

sample_ls = var["#SampleID"].unique()
sample_ls.sort()
print(sample_ls)

#Calulate the percentage of mut reads for total trf type.
proj_name = 'test'

result_df =pd.DataFrame(columns=['sample','s_desc','trf_type','mut_reads','no_mut_reads','total_reads','ratio'])
for s in sample_ls:
    trf_file = wdir+'/'+s+'_'+proj_name+"_hit.tab"
    s_desc = d["sample_dic"][s]
    TRF = pd.read_csv(trf_file, sep="\t", index_col=False)
    dic = {s:{'sample':s, 's_desc':s_desc,'data':{}}}
    for index, row in TRF.iterrows():
        read_num = int(row['mean_number'])
        trf_type = row['TRF_type']
        infor_contents  =row['brief_mapping_infor'].split(',')
        if not 'total' in dic[s]['data']:
            dic[s]['data']['total']={'mut_reads':0, 'no_mut_reads':0, 'total_reads':0, 'ratio':'0'}
        if not trf_type in dic[s]['data']:
            dic[s]['data'][trf_type]={'mut_reads':0, 'no_mut_reads':0, 'total_reads':0, 'ratio':'0'}
        dic[s]['data']['total']['total_reads']+=read_num
        dic[s]['data'][trf_type]['total_reads']+=read_num
        if infor_contents[4]==infor_contents[5]:
            dic[s]['data']['total']['no_mut_reads']+=read_num
            dic[s]['data'][trf_type]['no_mut_reads']+=read_num
        else:
            dic[s]['data']['total']['mut_reads']+=read_num
            dic[s]['data'][trf_type]['mut_reads']+=read_num
    for t in dic[s]['data']:
        row = {
            'sample':dic[s]['sample'],
            's_desc':dic[s]['s_desc'],
            'trf_type':t,
            'mut_reads':dic[s]['data'][t]['mut_reads'],
            'no_mut_reads':dic[s]['data'][t]['no_mut_reads'],
            'total_reads':dic[s]['data'][t]['total_reads'],
            'ratio':round(dic[s]['data'][t]['mut_reads']/dic[s]['data'][t]['total_reads'],3)
        }
        result_df = result_df.append(row,ignore_index=True)

csv_download_link(result_df, 'var_stat', False)
            
#Calulate the matrix of mutation across samples.
mut_read_cutoff = 200
mut_ratio_cutoff =0.8

var = var.loc[var['mut_reads']>mut_read_cutoff]
#sns.distplot(var['ratio'])
var_group= var.groupby(['#SampleID', 'family','loc','ref']).sum()
var_c_group= var.groupby(['#SampleID', 'family','loc','ref']).count()
var_group['ratio_10']= var_group['mut_reads']/var_group['total_reads']
sns.distplot(var_group['ratio_10'])

#var_group['mut_id']= var_group['family']+var_group['loc'].map(str)
var_group['mut_number'] = var_c_group['ratio']
var_10 = var_group.loc[var_group['ratio_10']>mut_ratio_cutoff]
#print(var_10)
print(var_10.xs('SRR1836123',level='#SampleID', drop_level=False))
mut_dic = {}


sample_ls=['SRR1836123',  'SRR1836124','SRR1836125',  'SRR1836126']
#sample_ls=['SRR1836125',  'SRR1836126']
#sample_ls=['SRR1836127',  'SRR1836128',  'SRR1836129',  'SRR1836130']

mut_matrix = pd.DataFrame(None, columns=sample_ls)
mut_ls = []
for index, row in var_10.iterrows():
    s_id = index[0]
    if not s_id in mut_dic:
        mut_dic[s_id]={}
    mut_id = index[1]+":"+str(index[2])+":"+index[3]
    if mut_id not in mut_ls:
        mut_ls.append(mut_id)
    if not mut_id in mut_dic[s_id]:
        mut_dic[s_id][mut_id]={}
    mut_dic[s_id][mut_id]={'r':row['ratio_10'],'n':row['mut_number']}

#Fill the matrix
#print(mut_ls)
sel_mut_ls = []
for m in mut_ls:
    row={}
    for s in sample_ls:
        if not m in mut_dic[s]:
            row[s]=0
        else:
            row[s]=mut_dic[s][m]['r']
    '''
    if ((row[sample_ls[0]]>mut_ratio_cutoff and row[sample_ls[1]]>mut_ratio_cutoff) or 
        (row[sample_ls[2]]>mut_ratio_cutoff and row[sample_ls[3]]>mut_ratio_cutoff)):
        mut_matrix=mut_matrix.append(row, ignore_index=True)
        sel_mut_ls.append(m)
    '''
    mut_matrix=mut_matrix.append(row, ignore_index=True)
    sel_mut_ls.append(m)
mut_matrix['mut']=sel_mut_ls
mut_matrix = mut_matrix.set_index('mut',drop=True, append=False, inplace=False, verify_integrity=False)
#print(mut_matrix)
    

csv_download_link(mut_matrix, 'var_group', False)

figure = d["report_dir"]+"var_matrix.png"
cm.drawExpressionClusterMatrix(mut_matrix, 
                               figure,
                               yaxis_font_size=7,
                               fig_width=8, 
                               fig_height=25)

In [3]:
from collections import OrderedDict
import pandas as pd

wdir = '/media/hqyone/3T/trna/data/celllines/tap'
dict1 = OrderedDict(sorted(d["sample_dic"].items())) 
# print(dict1)

for s, s_des in dict1.items():
    tab_file=wdir+"/"+s+"_test_hit.tab"
    df = pd.read_csv(tab_file, sep="\t", index_col=False).fillna('')
    hit_df= dl.add_aa_column(df, trna_id="tRNA_id")
    # .str.contains('Ju')
    aa_hit_df = hit_df[hit_df['read_3_fragment'].str.contains('AAAAAAA')]
    cca_aa_hit_df = aa_hit_df[aa_hit_df['read_fragment'].str.endswith('CCA')]
    #print(aa_hit_df['read_3_fragment'])
    cca_aa_reads = cca_aa_hit_df['mean_number'].sum()
    aa_reads = aa_hit_df['mean_number'].sum()
    noaa_hit_df = hit_df[~hit_df['read_3_fragment'].str.contains('AAAAAAA')]
    noaa_reads = noaa_hit_df['mean_number'].sum()
    print("######"+s_des)
    print(str(int(cca_aa_reads))+":"+str(int(aa_reads))+":"+str(int(noaa_reads))+":"+str(int(aa_reads+noaa_reads)))
    print(s_des+":"+str(round(aa_reads/hit_df['mean_number'].sum(),3)))
    # print(hit_df.columns)

FileNotFoundError: [Errno 2] File /media/hqyone/3T/trna/data/celllines/tap/SRR1836123_test_hit.tab does not exist: '/media/hqyone/3T/trna/data/celllines/tap/SRR1836123_test_hit.tab'

In [ ]:
'''
    The function shows the 5' and 3' base addition modifications of tRFs.
    The function will print the top 5' and 3' addition modifications
    And draw four types/columns of pie charts:
        1. Pie charts showing the ratio of tRFs with or without 5'-Addition modification
        2. Pie charts showing the composition of 5'-Addition modifications
        3. Pie charts showing the ratio of tRFs with or without 3'-Addition modification
        4. Pie charts showing the composition of 3'-Addition modifications
    @param d: The data object generated by data_loader.py
    @param proj_name: the name of project, the default is "test", use can check the name of hit.tab file
            it follows the pattern : <sampleID>+"_"+<proj_name>+"_hit.tab"
    @param top_num: The number of top addition modifications to be printed out
    @param radius: the radius of pies
    @param fontsize:
    @param fig_width:
    @return: None
'''
# rs.getAddModificationStatisitc(d, 'test')

# import pandas as pd
# import matplotlib.pyplot as plt
# pd.options.plotting.backend = "plotly"
# 
# proj_name ="test"
# sample_dic = d["sample_dic"]
# s_num =len(sample_dic.keys())
# 
# sample_ls = list(sample_dic.keys())
# sample_ls.sort()
# 
# 
# fig, axs = plt.subplots(s_num, 4, figsize=[14, s_num*3+2])
# radius = 1.2
# fontsize =14
# index = 0
# for s in sample_ls:
#     des = sample_dic[s]
#     hit_tab = d['wdir']+"/"+s+"_"+proj_name+"_hit.tab"
#     df = pd.read_csv(hit_tab, sep="\t")
# 
#     df = dl.add_aa_column(df, trna_id="tRNA_id")
# 
#     df['temp5'] = df['read_5_fragment']
#     group_a = df.groupby('read_5_fragment').sum()
#     axs[index,1].pie(group_a["mean_number"],labels=group_a.index,autopct='%1.1f%%',shadow=False, radius=radius,labeldistance=None)
#     print("\n"+s+"_"+des)
#     sorted_df =group_a.sort_values(["mean_number"], ascending=[False]).head(10)
#     with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also  
#         print(sorted_df['mean_number'])
#     group_c = df.groupby('read_3_fragment').sum()
#     sorted_df =group_c.sort_values(["mean_number"], ascending=[False]).head(10)
#     with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also  
#         print(sorted_df['mean_number'])
#     axs[index,3].pie(group_c["mean_number"],labels=group_c.index,autopct='%1.1f%%',shadow=False, radius=radius, labeldistance=None)
#     #axs[index,1].text(-0.06, 0.5, des, transform=axs[index,0].transAxes,va='center', fontsize=12, weight='bold', rotation=90)
# 
#     df.loc[df['temp5'].isna(),'read_5_fragment']='-'
#     df.loc[df['temp5'].notna(),'read_5_fragment']='Addition'
#     group_b = df.groupby('read_5_fragment').sum()
#     axs[index,0].pie(group_b["mean_number"],labels=group_b.index,autopct='%1.1f%%',shadow=False, radius=radius)
#     axs[index,0].text(-0.1, 0.5, des, transform=axs[index,0].transAxes,va='center', fontsize=fontsize, weight='bold', rotation=90)
# 
#     df['temp3'] = df['read_3_fragment']                                                     
#     df.loc[df['temp3'].isna(),'read_3_fragment']='-'
#     df.loc[df['temp3'].notna(),'read_3_fragment']='Addition'
#     group = df.groupby('read_3_fragment').sum()
#     axs[index,2].pie(group["mean_number"],labels=group.index,autopct='%1.1f%%',shadow=False, radius=radius)
#     if index==0:
#         axs[index,0].text(0.2, 1.1, "5'_Addition_Ratio", transform=axs[index,0].transAxes,va='center', fontsize=fontsize, weight='bold', rotation=0)
#         axs[index,1].text(1.4, 1.1, "5'_Addition_Dist", transform=axs[index,0].transAxes,va='center', fontsize=fontsize, weight='bold', rotation=0)
#         axs[index,2].text(2.6, 1.1, "3'_Addition_Ratio", transform=axs[index,0].transAxes,va='center', fontsize=fontsize, weight='bold', rotation=0)
#         axs[index,3].text(3.9, 1.1, "3'_Addition_Dist", transform=axs[index,0].transAxes,va='center', fontsize=fontsize, weight='bold', rotation=0)
#     index+=1

In [ ]:
'''
    The function shows the 5' and 3' base addition modifications of tRFs.
    The function will print the top 5' and 3' addition modifications
    And draw four types/columns of pie charts:
        1. Pie charts showing the ratio of tRFs with or without 5'-Addition modification
        2. Pie charts showing the composition of 5'-Addition modifications
        3. Pie charts showing the ratio of tRFs with or without 3'-Addition modification
        4. Pie charts showing the composition of 3'-Addition modifications
    @param d: The data object generated by data_loader.py
    @param proj_name: the name of project, the default is "test", use can check the name of hit.tab file
            it follows the pattern : <sampleID>+"_"+<proj_name>+"_hit.tab"
    @param top_num: The number of top addition modifications to be printed out
    @param radius: the radius of pies
    @param fontsize: font size of the title 
    @param fig_width: the width of the figure
    @param fig_height: the height of the figure
    @return: None
'''
rs.getAddModificationStatisitc2(d, 
                                proj_name = 'test', 
                                top_num=10,
                                fontsize=12,
                                fig_width=600,
                                fig_height=1200)

In [ ]:
from collections import OrderedDict 
import re
#print(df.columns)

a="GCAACGTCACGCGGGAGCCAAAAAAAAAA"
i ='NNNNCG'
p = "^"+i.replace('N','\w')

",aaa"
print(",aas".split(','))
print(p)
#p = '\w\w\w\wCG'
b = re.search(p, a)
if b: 
    print(b.group(0))
    print(b.span()[0])
    print (a[b.span()[1]:])

r_patterm="AAAAAAAAA"
p = "CCA"+r_patterm.replace('N','\w')
b = re.search(p, a)
if b:
    seq =a[:b.span()[0]+3]
    print(seq)

dict1 = OrderedDict(sorted(d["sample_dic"].items())) 
# print(dict1)
top_num = 5
for s, s_des in dict1.items():
    tab_file=wdir+"/"+s+"_test_hit.tab"
    df = pd.read_csv(tab_file, sep="\t", index_col=False).fillna('')
    print(df.head(10)[['read_5_fragment','read_fragment']])
    
    
    hit_df=dl.add_aa_column(df, trna_id="tRNA_id")
    read_5_num = float(hit_df.loc[hit_df['read_5_fragment']!=""]['mean_number'].sum())
    total_read_num= float(hit_df['mean_number'].sum())
    print(s_des+":"+s+":"+str(read_5_num/total_read_num))
    
    aa_group_df = aa_hit_df.groupby('read_5_fragment').sum()
    amio_sorted_df = aa_group_df.sort_values(["mean_number"], ascending=[False]).head(top_num)
    #print(amio_sorted_df['mean_number'])

In [ ]:
import time
def filterFastQ2FastA(fastq, filtered_fasta, num_dic_txt, qcutoff=0, num_cutoff=50):
    FASTQ  = open(fastq,'r')
    iFASTQ = iter(FASTQ)
    dic = {}
    name_dic = {}
    Filterecd_FASTA = open(filtered_fasta, 'w')
    NumDicFile = open(num_dic_txt,"w")
    total_num = 0
    low_quality_num=0
    high_quality_num = 0
    for line in iFASTQ:
        line  = line.strip()
        if line.startswith('@'):
            total_num+=1
            title = line.replace(" ","_")
            seq = next(iFASTQ).strip()
            title2 = next(iFASTQ).strip()
            quanlity = next(iFASTQ).strip()
            low_quanlity = False
            if total_num % 2000000==0:
                print("reads ("+str(total_num)+")")
                print("dic ("+str(len(dic.keys()))+")")
                

            # for i in quanlity:
            #     if ord(i)<=qcutoff+33:  #Based on Phred 33
            #         low_quanlity = True
            #         break
            if not low_quanlity:
                high_quality_num+=1
                if seq not in dic:
                    dic[seq] = {
                        "title":title,
                        #"seq":seq,
                        "count":1
                    }
                    name_dic[seq] = title
                else:
                    dic[seq]["count"]+=1
            else:
                low_quality_num+=1
    FASTQ.close()
    non_redundent_num = 0
    for seq in dic:
        title = name_dic[seq]
        if dic[seq]["count"]>=num_cutoff:
            Filterecd_FASTA.write(">" + dic[seq]["title"] + "\n")
            Filterecd_FASTA.write(seq + "\n")
            non_redundent_num += 1
            NumDicFile.write(title+"\t"+str(dic[seq]["count"])+"\t"+seq+"\n")
    NumDicFile.close()
    Filterecd_FASTA.close()
    return({"total_num":int(total_num),
          "removed_num":int(low_quality_num),
          "survived_num":int(high_quality_num),
          "non_redundent_num":int(non_redundent_num)})
# get starting time
start = time.time()

fastq = "/Users/hqyone/GitHub/rep_project/data/ngs_datasource/RNA_Seq_test/fastq/TestData/WT_1/ERX424842/ERR458495.fastq"
filtered_fasta = "/Users/hqyone/GitHub/rep_project/data/ngs_datasource/RNA_Seq_test/fastq/TestData/WT_1/ERX424842/ERR458495_f.fastq"
num_dic_txt = "/Users/hqyone/GitHub/rep_project/data/ngs_datasource/RNA_Seq_test/fastq/TestData/WT_1/ERX424842/ERR458495.dic.txt"
print(filterFastQ2FastA(fastq,filtered_fasta,num_dic_txt))

elapsed_time_fl = (time.time() - start) 
print(elapsed_time_fl)

In [ ]:
# processing the AA tail
from collections import OrderedDict 
import re


dict1 = OrderedDict(sorted(d["sample_dic"].items())) 
# print(dict1)

s_df = pd.DataFrame([], columns=['id','des','aa_ratio','cca_aa_reads','aa_reads','noaa_reads','total_reads'])

for s, s_des in dict1.items():
    tab_file=wdir+"/"+s+"_test_hit.tab"
    df = pd.read_csv(tab_file, sep="\t", index_col=False).fillna('')
    
    hit_df=dl.add_aa_column(df, trna_id="tRNA_id")
    hit_df=dl.add_AA_statistic(hit_df, column="read_3_fragment")
    #print(hit_df.columns)
    
    top_num=10
    hit_df=hit_df[hit_df['AA_tail']!=""][hit_df['AA_length']>7] #[hit_df['TRF_type']!="full_tRNA"]
    group_a = hit_df.groupby('AA_length').sum()
    sorted_df = group_a.sort_values(["mean_number"], ascending=[False]).head(top_num)
    print(sorted_df["mean_number"])
    
    # .str.contains('Ju')
    aa_hit_df = hit_df[hit_df['read_3_fragment'].str.contains('AAAAAAA')]
    aa_group_df = aa_hit_df.groupby('TRF_type').sum()
    
    aa_amino_group_df = aa_hit_df.groupby('AA').sum()
    amio_sorted_df = aa_amino_group_df.sort_values(["mean_number"], ascending=[False]).head(top_num)
    print(amio_sorted_df["mean_number"])
    
    cca_aa_hit_df = aa_hit_df[aa_hit_df['read_fragment'].str.endswith('CCA')]
    #print(aa_hit_df['read_3_fragment'])
    cca_aa_reads = cca_aa_hit_df['mean_number'].sum()
    aa_reads = aa_hit_df['mean_number'].sum()
    noaa_hit_df = hit_df[~hit_df['read_3_fragment'].str.contains('AAAAAAA')]
    noaa_reads = noaa_hit_df['mean_number'].sum()
    print("######"+s_des)
    print(aa_group_df['mean_number'])
    print(str(int(cca_aa_reads))+":"+str(int(aa_reads))+":"+str(int(noaa_reads))+":"+str(int(aa_reads+noaa_reads)))
    total_reads = aa_reads+noaa_reads
    aa_ratio = round(aa_reads/hit_df['mean_number'].sum(),3)
    print(s_des+":"+str(round(aa_ratio,3)))
    s_df = s_df.append({
        'id':s,
        'des':s_des,
        'cca_aa_reads':int(cca_aa_reads),
        'aa_reads':int(aa_reads),
        'noaa_reads':int(noaa_reads),
        'total_reads':int(total_reads),
        'aa_ratio': aa_ratio
    }, ignore_index=True)
print(s_df)
    # print(hit_df.columns)


In [ ]:
import time
import gc
import os
def filterFastQ2FastA(fastq, filtered_fasta, num_dic_txt, qcutoff=0, num_cutoff=50):
    big_file = False
    if os.path.getsize(fastq)>3259059200:
        big_file=True
    FASTQ  = open(fastq,'r')
    Filterecd_FASTA = open(filtered_fasta, 'w')
    NumDicFile = open(num_dic_txt,"w")
    dic = {}
    name_dic = {}
    total_num = 0
    low_quality_num=0
    high_quality_num = 0
    line_index=0
    seq=""
    title=""
    title2=""
    quanlity=""
    for line in FASTQ:
        line  = line.strip()
        line_index+=1
        if line_index%4==2:
            seq = line
        elif line_index%4==3:
            continue
            # title2 = line.replace(" ","_")
        elif line_index%4==0:
            quanlity = line
        elif line.startswith('@') and line_index%4==1 and seq!="":
            low_quanlity = False
            title=line.replace(" ","_")
            if total_num % 400000==0:
                print("reads ("+str(total_num)+")")
                keys = list(dic.keys())
                if big_file:
                    for k in keys:
                        if (dic[k]["count"]==1):
                            del dic[k]
                print("dic ("+str(len(dic.keys()))+")")
                gc.collect()
            # for i in quanlity:
            #     if ord(i)<=qcutoff+33:  #Based on Phred 33
            #         low_quanlity = True
            #         break
            if not low_quanlity :
                high_quality_num+=1
                if seq not in dic:
                    dic[seq] = {
                        "title":title,
                        #"seq":seq,
                        "count":1
                    }
                    name_dic[seq] = title
                else:
                    dic[seq]["count"]+=1
            total_num+=1
    FASTQ.close()
    non_redundent_num = 0
    for seq in dic:
        title = name_dic[seq]
        if dic[seq]["count"]>=num_cutoff:
            Filterecd_FASTA.write(">" + dic[seq]["title"] + "\n")
            Filterecd_FASTA.write(seq + "\n")
            non_redundent_num += 1
            NumDicFile.write(title+"\t"+str(dic[seq]["count"])+"\t"+seq+"\n")
    NumDicFile.close()
    Filterecd_FASTA.close()
    return({"total_num":int(total_num),
          "removed_num":int(total_num-high_quality_num),
          "survived_num":int(high_quality_num),
          "non_redundent_num":int(non_redundent_num)})
# get starting time
start = time.time()

fastq = "/Users/hqyone/GitHub/rep_project/data/ngs_datasource/RNA_Seq_test/fastq/TestData/WT_1/ERX424842/ERR458495.fastq"
filtered_fasta = "/Users/hqyone/GitHub/rep_project/data/ngs_datasource/RNA_Seq_test/fastq/TestData/WT_1/ERX424842/ERR458495_f.fastq"
num_dic_txt = "/Users/hqyone/GitHub/rep_project/data/ngs_datasource/RNA_Seq_test/fastq/TestData/WT_1/ERX424842/ERR458495.dic.txt"
print(filterFastQ2FastA(fastq,filtered_fasta,num_dic_txt))

elapsed_time_fl = (time.time() - start) 
print(elapsed_time_fl)

